# **Chargement des données**

In [ ]:
# Charge les packages
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Dense, Conv2D, Flatten, MaxPool2D
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
# Charge le FASHION-MNIST dataset
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()

X_train = X_train.astype('float32') / 255
X_test = X_test.astype('float32') / 255

X_train = np.expand_dims(X_train, axis= -1)
X_test = np.expand_dims(X_test, axis= -1)

y_train_OneHot = to_categorical(y_train)
y_test_OneHot = to_categorical(y_test)

In [ ]:
# Affiche quelques informations sur le dataset
n_class = len(np.unique(y_train))
print(f"Il y a {n_class} types de vêtements différent dans le dataset")
print(f"Il y a {X_train.shape[0]} exemples dans le dataset d'entrainement")
print(f"Format des images : {X_train.shape[1:]} pixels")

In [ ]:
ids_to_label = {
    0 : 't-shirt / haut',
    1 : 'pantalon',
    2 : 'pull',
    3 : 'robe',
    4 : 'manteau',
    5 : 'sandale',
    6 : 'chemise',
    7 : 'sneaker',
    8 : 'sac',
    9 : 'chaussures'
}

# **Définition de fonctions pour visualiser et monitorer**

In [ ]:
# Fonction pour afficher une image
def display_image(img_number: int, dataset= 'train') :
  if dataset == "train" :
    img = X_train[img_number, :, :]
    label = ids_to_label[y_train[img_number]]

  elif dataset == 'test' :
    img = X_test[img_number, :, :]
    label = ids_to_label[y_test[img_number]]

  plt.figure(figsize= (5,5))
  plt.imshow(img, cmap= 'gray');
  plt.title(str(label))
  plt.axis('off')
  plt.show();

  return None

In [ ]:
display_image(1)

In [ ]:
# Fonction pour monitorer l'entrainement du modèle
def display_training(history) :
  train_acc = history.history['accuracy']
  val_acc = history.history['val_accuracy']
  x_axis= np.arange(len(train_acc))

  plt.figure(figsize= (8,6))
  plt.plot(x_axis, train_acc, c= 'b', label= 'Training accuracy')
  plt.plot(x_axis, val_acc, c= 'g', label= 'Validation accuracy')
  plt.title('Training and validation accuracy')
  plt.xlabel('Epochs')
  plt.ylabel('Accuracy en %')
  plt.legend()
  plt.show();

In [ ]:
# Fonction pour visualiser les prédictions du modèle
def display_predictions(correct_or_error, seed= 42) :

  np.random.seed(seed)
  y_pred = model.predict(X_test)
  y_pred = np.argmax(y_pred, axis= 1)

  if correct_or_error == 'correct' :
    mask = (y_pred == y_test)
  elif correct_or_error == 'error' :
    mask = (y_pred != y_test)

  y_pred_restricted = y_pred[mask]
  y_test_restricted = y_test[mask]
  X_test_restricted = X_test[mask, :, :]

  idx = np.random.choice(a= len(y_pred_restricted), size= 8, replace= False)
  pred_labels = [ids_to_label[y_pred_restricted[i]] for i in idx]
  test_labels = [ids_to_label[y_test_restricted[i]] for i in idx]

  plt.figure(figsize= (12, 6))
  for i in range(2) :
    for j in range(4) :
      plt.subplot(2, 4, 4*i+j+1)
      plt.imshow(X_test_restricted[idx[4*i+j], :, :], cmap= 'gray')
      plt.title(f'Prédiction : {pred_labels[4*i+j]}\nVrai : {test_labels[4*i+j]}')
      plt.axis('off')

  return None

# **Création et entrainement du modèle**

In [ ]:
# Définit l'architecture du Réseau de Neurones
class NeuralNetwork(tf.keras.Model) :
  def __init__(self, n_hidden= 1, hidden_size = [64], n_class= n_class) :
    super().__init__()

    self.n_class = n_class
    self.n_hidden = n_hidden

    assert n_hidden == len(hidden_size), \
    f'ATTENTION, "n_hidden" indique {n_hidden} couches dans le modèle mais seules {len(hidden_size)} sont définies dans "hidden_size"'

    self.flatten = Flatten(input_shape= (28, 28))
    self.input_dense = Dense(units= 16, activation= 'relu')

    self.hidden_layer_list = list()
    for i in range(n_hidden) :
      layer = Dense(units= hidden_size[i], activation= 'relu')
      self.hidden_layer_list.append(layer)

    self.output_dense = Dense(units= n_class, activation= 'softmax')

  def call(self, inputs) :
    x = self.flatten(inputs)
    x = self.input_dense(x)
    for i in range(self.n_hidden) :
      x = self.hidden_layer_list[i](x)

    output = self.output_dense(x)

    return output

In [ ]:
# Paramètres utilisateurs
n_hidden= 1
hidden_size= [32]
n_epochs = 20
batch_size = 32

# Créer et initialise le modèle
model = NeuralNetwork(n_hidden= n_hidden,
                      hidden_size= hidden_size,
                      n_class= n_class)

model.compile(optimizer= 'adam',
              loss= 'categorical_crossentropy',
              metrics= ['accuracy'])

In [ ]:
callbacks = [
    EarlyStopping(monitor= 'val_accuracy',
                  patience= 3,
                  mode= 'max',
                  min_delta= 5e-4,
                  restore_best_weights= True)
    ]

# Entraine le modèle
history = model.fit(
    x= X_train,
    y= y_train_OneHot,
    batch_size= batch_size,
    epochs= n_epochs,
    validation_data= (X_test, y_test_OneHot),
    callbacks= callbacks
    )

print(f'\nLe modèle a {model.evaluate(X_test, y_test_OneHot)[1] *100:.0f}% de bonnes réponses sur le dataset de test')

In [ ]:
# Visualisation de l'entrainement du modèle
display_training(history)

In [ ]:
# Visualisation des exemples correctement classifiés par le modèle
display_predictions(correct_or_error= 'correct', seed= 123)

In [ ]:
# Visualisation des exemples mal classifiés par le modèle
display_predictions(correct_or_error= 'error', seed= 123)

# **Test avec un Convolution Network**

In [ ]:
# Définit l'architecture du Réseau de Neurones
class ConvNetwork(tf.keras.Model) :
  def __init__(self, n_class= n_class) :
    super().__init__()

    self.conv1 = Conv2D(filters= 16, kernel_size= (3,3), padding= 'same', activation= 'relu')
    self.pool1 = MaxPool2D(pool_size= (2,2), strides= (2,2))
    self.conv2 = Conv2D(filters= 32, kernel_size= (3,3), padding= 'same', activation= 'relu')
    self.pool2 = MaxPool2D(pool_size= (2,2), strides= (2,2))

    self.flatten = Flatten()
    self.dense1 = Dense(units= 32)
    self.output_dense = Dense(units= n_class, activation= 'softmax')

  def call(self, inputs) :
    x = self.conv1(inputs)
    x = self.pool1(x)
    x = self.conv2(x)
    x = self.pool2(x)
    x = self.flatten(x)
    x = self.dense1(x)
    output = self.output_dense(x)

    return output

In [ ]:
model = ConvNetwork(n_class= n_class)

model.compile(optimizer= 'adam',
              loss= 'categorical_crossentropy',
              metrics= ['accuracy'])

# Entraine le modèle
history = model.fit(
    x= X_train,
    y= y_train_OneHot,
    batch_size= batch_size,
    epochs= n_epochs,
    validation_data= (X_test, y_test_OneHot),
    callbacks= callbacks
    )

print(f'\nLe modèle a {model.evaluate(X_test, y_test_OneHot)[1] *100:.0f}% de bonnes réponses sur le dataset de test')

In [ ]:
# Visualisation de l'entrainement du modèle
display_training(history)